In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
class Benchmark(object):
    
    def read_data(csv_path):
        try:
            data = pd.read_csv(csv_path)
            data.columns = ['category','year','market_channel','department',
                'roi_range','number_of_tpr_weeks','number_of_display_weeks',
                'number_of_feature_weeks','number_of_event_weeks','mfr_roi_value',
                'trade_direct_promo_spend','number_of_weeks','PPI','BP',
                'DISO','FEAO','FEDI']
            
        except ValueError:
            print('File is either unavailable or non-existent.')
        
        return data

    def find_roi(in_data,agg_var='category',year=2015):
        roi_data = in_data[[agg_var,'year','roi_range','mfr_roi_value','trade_direct_promo_spend']]
        roi_data = roi_data[(roi_data.year==year) & (roi_data.roi_range.notnull())]
        roi_pivot = pd.pivot_table(roi_data,values=['mfr_roi_value','trade_direct_promo_spend'], 
                                   index=[agg_var],aggfunc='sum')

        roi_pivot['roi'] = (roi_pivot['mfr_roi_value']/roi_pivot['trade_direct_promo_spend']) + 1
        
        return roi_pivot.drop(roi_pivot.columns[[0,1]],axis=1)
        
    def find_coeff(in_data,agg_var,coeff):
        return pd.pivot_table(in_data,values=[coeff],index=[agg_var],aggfunc='mean')
        
    def graph(formula,x_range):
        x = np.array(x_range)
        y = eval(formula)
        plt.plot(x,y)
        
    def coeff_roi_reg(in_data,agg_var,year,norms_coeff,path='none'):
        coeff_roi_data = pd.merge(Benchmark.find_roi(in_data,agg_var,year),
                                  Benchmark.find_coeff(in_data,agg_var,norms_coeff),
                                  left_index=True, right_index=True)
        coeff_roi_data = coeff_roi_data[coeff_roi_data['roi'].notnull() & coeff_roi_data[norms_coeff].notnull()]
        
        roi = coeff_roi_data['roi']
        coeff = sm.add_constant(coeff_roi_data[norms_coeff])
        
        model = sm.OLS(roi,coeff)
        results = model.fit()
        #find equation for graph
        intercept = round(results.params[0],3)
        coeff_effect = round(results.params[1],3)
        trendline = str(intercept)+'+x*'+str(coeff_effect)
        r2 = round( results.rsquared, 3)
        
        plt.plot(coeff[norms_coeff],roi,'o')
        
        Benchmark.graph(trendline,coeff[norms_coeff])
        
        plt.xlabel(norms_coeff)
        plt.ylabel('ROI')
        plt.title('Promo ROI vs ' + norms_coeff + ' by ' + agg_var)        
        plt.text(min(coeff[norms_coeff]),max(roi)-.3,'ROI ~ ' + str(intercept) + str(coeff_effect) + '*' + norms_coeff 
                 + '\n' + 'R-Squared: '+str(r2))
        if(path=='none'):
            plt.show()
        else:
            plt.savefig(path)